In [ ]:
import pandas as pd
import openai
import time
import os
from tqdm import tqdm


In [ ]:
# Verify OpenAI library version
print(f"OpenAI library version: {openai.__version__}")  # Should print 0.28.0

In [ ]:
# Set OpenAI API key
openai.api_key = 'sk-proj-dvUlhrqVeqGmdn2a7b3pT3BlbkFJbWEX2j67TZIrKSZvCfzm'

In [ ]:
# Construct the file path
base_dir = r'C:\Users\monbi\OneDrive\Documents\BSE\Term 3\Masters Thesis\ECB_Perceived_Cacophony'
file_name = 'Articles_to_Score_3000.xlsx'
file_path = os.path.join(base_dir, file_name)
print("Loading the Excel file...")
articles_df = pd.read_excel(file_path)
print("Excel file loaded successfully.")

# Display the head of the DataFrame to check the data
print("Head of the loaded DataFrame:")
articles_df.head()



In [ ]:
print("Shape of the DataFrame:", articles_df.shape)

In [ ]:
# Define the classify and score function
def classify_and_score_article(article, prompt):
    while True:
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4",
                messages=[
                    {"role": "system", "content": "You are a financial expert."},
                    {"role": "user", "content": prompt + article}
                ],
                max_tokens=150  # Adjust max_tokens if needed
            )
            result = response['choices'][0]['message']['content'].strip()

            # Assign scores based on textual labels
            if "HAWKISH" in result.upper():
                score = 1
            elif "DOVISH" in result.upper():
                score = -1
            elif "NEUTRAL" in result.upper():
                score = 0
            else:
                raise ValueError("Incomplete response from API")
                
            return score

        except openai.error.RateLimitError:
            print("Rate limit exceeded. Retrying in 10 seconds...")
            time.sleep(10)
        except openai.error.APIError as e:
            print(f"API error: {e}. Retrying in 10 seconds...")
            time.sleep(10)
        except Exception as e:
            print(f"Unexpected error: {e}. Skipping this article.")
            return "Error"


In [ ]:
# Define the best prompt
best_prompt = "Discard all the previous instructions. Behave like you are an expert economist. Evaluate the following statement from an ECB member from the perspective of how the media reader might interpret it. Would it likely be perceived as HAWKISH (signaling higher rates), DOVISH (signaling lower rates), or NEUTRAL (ambiguous): "

# Score each article using the best model prompt
print("Classifying and scoring articles using OpenAI_Prompt_4 for the complete dataset...")

# Use tqdm to add a progress bar to the apply function
tqdm.pandas(desc="Scoring Articles")
articles_df['OpenAI_Score'] = articles_df['Manual.summary'].progress_apply(lambda article: classify_and_score_article(article, best_prompt))

# Keep only the necessary columns
output_df = articles_df[['Manual.summary', 'OpenAI_Score']]

# Display the first few rows of the updated DataFrame
print("Classification and scoring completed. Here are the first few results:")
print(output_df.head())


In [ ]:
# Save the results to a new Excel file
output_file_name = 'Scored_Articles_3000.xlsx'
output_file = os.path.join(base_dir, output_file_name)
print(f"Saving the results to {output_file}...")
output_df.to_excel(output_file, index=False)
print(f"Results saved to {output_file}")

# Reload the saved file to verify its contents
print("Reloading the saved Excel file to verify its contents...")
verified_df = pd.read_excel(output_file)
print("Head of the reloaded DataFrame:")
print(verified_df.head())
